In [1]:
import sys
from Util.util import marking_word_tag,marking_character_tag
from Util.File_Util import get_text_from_txt,get_json_info

from pprint import pprint
import os

work_path = './Corpus_Path/2024年03月03日15时01分'
index = 0
for filename in os.listdir(work_path):
    # print(filename)
    major_path = os.path.join(work_path, filename)
    pdf_path = os.path.join(major_path, filename) + ".pdf"
    txt_path = os.path.join(major_path, filename) + ".txt"
    json_path = os.path.join(major_path, filename) + ".json"
    # print(file_pdf)
    if not os.path.exists(pdf_path):
        print("pdf不存在")
        continue
    text = get_text_from_txt(txt_path, pdf_path)
    ner_json = get_json_info(json_path)
    # 获取词标注
    all_tag = marking_word_tag(ner_json, text, log_info=filename)
    # pprint(all_tag)
    char_tag = marking_character_tag(all_tag,text,major_path)
    print(char_tag)
    # 标注好的json文件
    index = index + 1

    if index >= 1:
        break


从./Corpus_Path/key_mapping.json读取成功
从./Corpus_Path/2024年03月03日15时01分\101\101.json读取成功
标记已写入./Corpus_Path/2024年03月03日15时01分\101\tag.txt
[['蔡', 'B-NM'], ['于', 'I-NM'], ['纬', 'I-NM'], ['求', 'O'], ['职', 'O'], ['意', 'O'], ['向', 'O'], ['：', 'O'], ['市', 'B-CO'], ['场', 'I-CO'], ['专', 'I-CO'], ['员', 'I-CO'], ['年', 'O'], ['龄', 'O'], ['：', 'O'], ['2', 'B-AG'], ['3', 'I-AG'], ['岁', 'O'], ['地', 'O'], ['址', 'O'], ['：', 'O'], ['广', 'B-LV'], ['东', 'I-LV'], ['省', 'I-LV'], ['广', 'I-LV'], ['州', 'I-LV'], ['市', 'I-LV'], ['电', 'O'], ['话', 'O'], ['：', 'O'], ['1', 'B-TL'], ['3', 'I-TL'], ['5', 'I-TL'], ['1', 'I-TL'], ['1', 'I-TL'], ['2', 'I-TL'], ['2', 'I-TL'], ['3', 'I-TL'], ['3', 'I-TL'], ['4', 'I-TL'], ['4', 'I-TL'], ['邮', 'O'], ['箱', 'O'], ['：', 'O'], ['s', 'B-EM'], ['e', 'I-EM'], ['r', 'I-EM'], ['v', 'I-EM'], ['i', 'I-EM'], ['c', 'I-EM'], ['e', 'I-EM'], ['@', 'I-EM'], ['5', 'I-EM'], ['0', 'I-EM'], ['0', 'I-EM'], ['d', 'I-EM'], ['.', 'I-EM'], ['m', 'I-EM'], ['e', 'I-EM'], ['教', 'O'], ['育', 'O'], ['背', 'O'

In [4]:
from transformers import BertTokenizer, BertModel

# 加载预训练的Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')  # 使用适用于中文的预训练模型
word_to_idx = []
# 对每个句子进行编码
for sent in text:
    # 编码，同时添加特殊标记
    encoded_input = tokenizer.encode(sent, add_special_tokens=True)
    print(f"句子: {sent}")
    print(f"编码: {encoded_input[1]}")
    idx = encoded_input[1]
    word_to_idx.append(idx)
#输出：[8, 35, 826, 52, 10, 159, 559, 98, 147, 18, 5, 7, 27, 59, 414, 12043]
print(word_to_idx)

句子: 蔡
编码: 5918
句子: 于
编码: 754
句子: 纬
编码: 5281
句子: 求
编码: 3724
句子: 职
编码: 5466
句子: 意
编码: 2692
句子: 向
编码: 1403
句子: ：
编码: 8038
句子: 市
编码: 2356
句子: 场
编码: 1767
句子: 专
编码: 683
句子: 员
编码: 1447
句子: 年
编码: 2399
句子: 龄
编码: 7977
句子: ：
编码: 8038
句子: 2
编码: 123
句子: 3
编码: 124
句子: 岁
编码: 2259
句子: 地
编码: 1765
句子: 址
编码: 1770
句子: ：
编码: 8038
句子: 广
编码: 2408
句子: 东
编码: 691
句子: 省
编码: 4689
句子: 广
编码: 2408
句子: 州
编码: 2336
句子: 市
编码: 2356
句子: 电
编码: 4510
句子: 话
编码: 6413
句子: ：
编码: 8038
句子: 1
编码: 122
句子: 3
编码: 124
句子: 5
编码: 126
句子: 1
编码: 122
句子: 1
编码: 122
句子: 2
编码: 123
句子: 2
编码: 123
句子: 3
编码: 124
句子: 3
编码: 124
句子: 4
编码: 125
句子: 4
编码: 125
句子: 邮
编码: 6934
句子: 箱
编码: 5056
句子: ：
编码: 8038
句子: s
编码: 161
句子: e
编码: 147
句子: r
编码: 160
句子: v
编码: 164
句子: i
编码: 151
句子: c
编码: 145
句子: e
编码: 147
句子: @
编码: 137
句子: 5
编码: 126
句子: 0
编码: 121
句子: 0
编码: 121
句子: d
编码: 146
句子: .
编码: 119
句子: m
编码: 155
句子: e
编码: 147
句子: 教
编码: 3136
句子: 育
编码: 5509
句子: 背
编码: 5520
句子: 景
编码: 3250
句子: 2
编码: 123
句子: 0
编码: 121
句子: 1
编码: 122
句子: 8
编码: 129
句子: .
编码: 119
句子: 9
编码: 130
句子

In [8]:
from transformers import BertTokenizer
import json

# 加载预训练的tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 获取tokenizer的词汇表
vocab = tokenizer.vocab
index_to_word = {}
word_to_index = {}
# 移除'##'并反转词汇表，使索引成为键，词汇成为值
for word,idx in vocab.items():
    word = word.replace("##", "")
    print(word,idx,len(word))
    if len(word) == 1 or idx == 0:
        index_to_word[idx] = word
        word_to_index[word] = idx
# （可选）将反转的词汇表保存为JSON文件
with open('./Corpus_Path/Vocal/idx_to_chinese.json', 'w', encoding='utf-8') as f:
    json.dump(index_to_word, f, ensure_ascii=False, indent=4)
with open('./Corpus_Path/Vocal/chinese_to_idx.json', 'w', encoding='utf-8') as f:
    json.dump(word_to_index, f, ensure_ascii=False, indent=4)

[PAD] 0 5
[unused1] 1 9
[unused2] 2 9
[unused3] 3 9
[unused4] 4 9
[unused5] 5 9
[unused6] 6 9
[unused7] 7 9
[unused8] 8 9
[unused9] 9 9
[unused10] 10 10
[unused11] 11 10
[unused12] 12 10
[unused13] 13 10
[unused14] 14 10
[unused15] 15 10
[unused16] 16 10
[unused17] 17 10
[unused18] 18 10
[unused19] 19 10
[unused20] 20 10
[unused21] 21 10
[unused22] 22 10
[unused23] 23 10
[unused24] 24 10
[unused25] 25 10
[unused26] 26 10
[unused27] 27 10
[unused28] 28 10
[unused29] 29 10
[unused30] 30 10
[unused31] 31 10
[unused32] 32 10
[unused33] 33 10
[unused34] 34 10
[unused35] 35 10
[unused36] 36 10
[unused37] 37 10
[unused38] 38 10
[unused39] 39 10
[unused40] 40 10
[unused41] 41 10
[unused42] 42 10
[unused43] 43 10
[unused44] 44 10
[unused45] 45 10
[unused46] 46 10
[unused47] 47 10
[unused48] 48 10
[unused49] 49 10
[unused50] 50 10
[unused51] 51 10
[unused52] 52 10
[unused53] 53 10
[unused54] 54 10
[unused55] 55 10
[unused56] 56 10
[unused57] 57 10
[unused58] 58 10
[unused59] 59 10
[unused60] 60 

In [2]:
from transformers import BertTokenizer

# 加载预训练的tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 获取tokenizer的词汇表
vocab = tokenizer.vocab

# 移除'##'并反转词汇表，使索引成为键，词汇成为值
index_to_word = {idx: word.replace("##", "") for word, idx in vocab.items()}

# 打印一些示例看看效果
for idx in range(20392, 20399):
    print(f"{idx}: {index_to_word[idx]}")


20392: 阖
20393: 阙
20394: 阚
20395: 阜
20396: 队
20397: 阡
20398: 阪


In [7]:
def decode_tagged_text(text, tagged_text):
    decoded_annotations = []
    current_annotation = None
    current_tag = None

    for char, tag in zip(text, tagged_text):
        if tag.startswith('B-'):
            # 如果当前已有标注，则先保存当前标注
            if current_annotation is not None:
                decoded_annotations.append((current_annotation, current_tag))
            # 开始新的标注
            current_annotation = char
            current_tag = tag[2:]  # 移除 'B-' 前缀以获取标记
        elif tag.startswith('I-') and current_annotation is not None:
            # 如果当前字符是标注的一部分，则添加到当前标注
            current_annotation += char
        else:
            # 当前字符不属于任何标注，保存并重置当前标注（如果有的话）
            if current_annotation is not None:
                decoded_annotations.append((current_annotation, current_tag))
                current_annotation = None
                current_tag = None

    # 确保最后一个标注被添加
    if current_annotation is not None:
        decoded_annotations.append((current_annotation, current_tag))

    return decoded_annotations

# 示例文本和标注
text = "This is an example."
tagged_text = ['O', 'O', 'O', 'O', 'B-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'I-TAG', 'O', 'O', 'O', 'O', 'O']

# 解码标注
decoded_annotations = decode_tagged_text(text, tagged_text)

# 打印解码结果
for annotation, tag in decoded_annotations:
    print(f"Annotation: {annotation}, Tag: {tag}")


Annotation:  is an , Tag: TAG
